### IMPORTS

In [1]:
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az
import warnings

warnings.filterwarnings('ignore')

WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


$\textbf{QUESTION 1}$

In [2]:
babies = pd.read_csv('babies.csv')
cord_clamped = babies['x']
not_clamped = babies['y']

In [3]:
print(cord_clamped.describe())

9.64375


In [4]:
print(not_clamped.describe())

12.09375


In [5]:

with pm.Model() as model:
    
    # Define noninformative Gamma priors
    shape_prior = 0.001
    rate_prior = 0.001
    alpha1 = pm.Gamma(name="alpha1", alpha=shape_prior, beta=rate_prior)
    beta1 = pm.Gamma(name="beta1", alpha=shape_prior, beta=rate_prior)
    alpha2 = pm.Gamma(name="alpha2", alpha=shape_prior, beta=rate_prior)
    beta2 = pm.Gamma(name="beta2", alpha=shape_prior, beta=rate_prior)
    
    # Priors for the means
    # mean1 = pm.Gamma(name='mean1', alpha=shape_prior, beta=rate_prior)
    # mean2 = pm.Gamma(name='mean2', alpha=shape_prior, beta=rate_prior)
    mean1, mean2 = alpha1 / beta1, alpha2 / beta2 

    # Likelihoods for the data
    likelihood1 = pm.Gamma(name='likelihood1', alpha=alpha1, beta=beta1, observed=cord_clamped)
    likelihood2 = pm.Gamma(name='likelihood2', alpha=alpha2, beta=beta2, observed=not_clamped)

    # Difference in means
    diff = mean1 - mean2
    diff_means = pm.Deterministic(name='diff_means', var=diff)

    # Sampling
    trace = pm.sample(draws=1000, tune=1000, target_accept=0.90, cores=None)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha1, beta1, alpha2, beta2]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 144 seconds.


In [6]:
# View trace diff_means
diff_means_trace = az.summary(data=trace, var_names=['diff_means'], hdi_prob=.90)
print(f"Trace Summary for difference in means:\n{diff_means_trace}\n")

# Print trace summary
print(f"Trace Summary:\n{az.summary(data=trace, hdi_prob=.90)}\n")

# Check Credible Set
print(f"The 90% Credible Set for the difference of means:\n{diff_means_trace}")

Trace Summary for difference in means:
             mean     sd  hdi_5%  hdi_95%  mcse_mean  mcse_sd  ess_bulk  \
diff_means -2.455  0.786  -3.736    -1.21      0.012    0.009    3983.0   

            ess_tail  r_hat  
diff_means    2723.0    1.0  

Trace Summary:
              mean      sd  hdi_5%  hdi_95%  mcse_mean  mcse_sd  ess_bulk  \
alpha1      35.020  12.358  16.270   54.387      0.374    0.265    1072.0   
beta1        3.633   1.291   1.467    5.472      0.039    0.028    1063.0   
alpha2      27.164   9.849  12.439   43.540      0.282    0.199    1141.0   
beta2        2.247   0.819   0.975    3.566      0.023    0.016    1185.0   
diff_means  -2.455   0.786  -3.736   -1.210      0.012    0.009    3983.0   

            ess_tail  r_hat  
alpha1        1224.0    1.0  
beta1         1246.0    1.0  
alpha2         898.0    1.0  
beta2          950.0    1.0  
diff_means    2723.0    1.0  

The 90% Credible Set for the difference of means:
             mean     sd  hdi_5%  hdi_95

$$
\text{The 90\% credible set doesn't contain 0.} \Rightarrow \text{The difference is statistically significant.}
$$

$\textbf{QUESTION 2}$

In [7]:
intraocular_pressure = pd.read_excel('iop2.xlsx', header=None, names=['indicator', 'cornea_thickness'])
low_iop = intraocular_pressure['indicator']
corn_thickness = intraocular_pressure['cornea_thickness']

In [8]:
print(low_iop.describe())

count    140.000000
mean       0.242857
std        0.430349
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: indicator, dtype: float64


In [9]:
print(corn_thickness.describe())

count    140.000000
mean     484.657143
std       39.032319
min      386.000000
25%      456.750000
50%      482.500000
75%      513.000000
max      590.000000
Name: cornea_thickness, dtype: float64


In [10]:
# Borrow some code from Aaron's GitHub
def standardize(x):
    
    mu = x.mean()
    sig = x.std()

    return (x - mu) / (2 * sig)

$$
\textbf{PART A}
$$

In [11]:
# Borrow some code from Aaron's GitHub
with pm.Model() as mod_logistic:

    # Define x, y
    corn_standard = standardize(corn_thickness)
    corn_data = pm.Data(name="corn_data", value=corn_standard, mutable=True)
    iop_data = pm.Data(name="iop_data", value=low_iop, mutable=False)

    # Define alpha, beta for logistic regression
    alpha = pm.Normal(name="alpha", mu=0, sigma=2)
    betas = pm.Normal(name="beta", mu=0, sigma=1)

    logist = alpha + pm.math.dot(l=corn_data, r=betas)
    p = pm.math.invlogit(logist)

    pm.Bernoulli(name="low_iop", p=p, observed=iop_data)

    trace = pm.sample(draws=1000, tune=1000, cores=None, chains=4)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 221 seconds.


In [12]:
# Summarize the trace
print(az.summary(data=trace, hdi_prob=.95))

        mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
alpha -1.320  0.219    -1.736     -0.883      0.004    0.003    3460.0   
beta  -1.763  0.443    -2.645     -0.942      0.008    0.006    3288.0   

       ess_tail  r_hat  
alpha    3018.0    1.0  
beta     2876.0    1.0  


$$
\textbf{PART B}
$$

In [13]:
# Borrow some code from Aaron's GitHub
with pm.Model() as mod_logistic:

    # Define x, y
    corn_data = 490
    iop_data = pm.Data(name="iop_data", value=low_iop, mutable=False)

    # Define alpha, beta for logistic regression
    alpha = pm.Normal(name="alpha", mu=0, sigma=2)
    betas = pm.Normal(name="beta", mu=0, sigma=1)

    logist = alpha + pm.math.dot(l=corn_data, r=betas)
    p = pm.math.invlogit(logist)

    pm.Bernoulli(name="low_iop", p=p, observed=iop_data)

    trace = pm.sample(draws=1000, tune=1000, cores=None, chains=4)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 60 seconds.


In [14]:
# Summarize the trace
print(az.summary(data=trace, hdi_prob=.95))

        mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
alpha  0.067  2.019    -3.823      4.063      0.086    0.068     556.0   
beta  -0.002  0.004    -0.011      0.006      0.000    0.000     552.0   

       ess_tail  r_hat  
alpha     508.0    1.0  
beta      513.0    1.0  


$$
\textbf{PART C}
$$

In [32]:
# Borrow some code from Aaron's GitHub
with pm.Model() as mod_probit:

    # Define x, y
    corn_standard = standardize(corn_thickness)
    corn_data = pm.Data(name="corn_data", value=corn_standard, mutable=True)
    iop_data = pm.Data(name="iop_data", value=low_iop, mutable=False)

    # Define alpha, beta for logistic regression
    alpha = pm.Normal(name="alpha", mu=0, sigma=2)
    betas = pm.Normal(name="beta", mu=0, sigma=1)

    logist = alpha + pm.math.dot(l=corn_data, r=betas)
    p = pm.math.invprobit(logist)

    pm.Bernoulli(name="low_iop", p=p, observed=iop_data)

    trace = pm.sample(draws=1000, tune=1000, cores=None, chains=4,
                      idata_kwargs=dict(log_likelihood=True))
    az.waic(data=trace, scale="deviance")

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 212 seconds.


In [33]:
# Summarize the trace
print(az.summary(data=trace, hdi_prob=.95))

        mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
alpha -0.826  0.132    -1.086     -0.576      0.002    0.002    3272.0   
beta  -1.221  0.291    -1.805     -0.659      0.005    0.004    2937.0   

       ess_tail  r_hat  
alpha    2617.0    1.0  
beta     2579.0    1.0  


In [34]:
# Borrow some code from Aaron's GitHub
with pm.Model() as mod_probit:

    # Define x, y
    corn_data = 490
    iop_data = pm.Data(name="iop_data", value=low_iop, mutable=False)

    # Define alpha, beta for logistic regression
    alpha = pm.Normal(name="alpha", mu=0, sigma=2)
    betas = pm.Normal(name="beta", mu=0, sigma=1)

    logist = alpha + pm.math.dot(l=corn_data, r=betas)
    p = pm.math.invprobit(logist)
    
    pm.Bernoulli(name="low_iop", logit_p=p, observed=iop_data)

    trace = pm.sample(draws=1000, tune=1000, cores=None, chains=4, 
                      idata_kwargs=dict(log_likelihood=True))
    az.waic(data=trace, scale="deviance")

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 69 seconds.
There were 252 divergences after tuning. Increase `target_accept` or reparameterize.


In [35]:
# Summarize the trace
print(az.summary(data=trace, hdi_prob=.95))

        mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
alpha  0.064  2.022    -3.696      4.306      0.056    0.048    1305.0   
beta  -0.793  0.594    -1.922     -0.004      0.017    0.012    1384.0   

       ess_tail  r_hat  
alpha    1173.0    1.0  
beta     1613.0    1.0  


$\textbf{QUESTION 3}$

$$
\textbf{PART A}
$$

$$
\textbf{PART B}
$$